In [307]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from joblib import dump, load

import numpy as np

In [370]:
#loading saved model
regressor = load('20220818_randomforest_model.joblib')

In [371]:
# Read player model and ages
xl1 = pd.read_csv('data_cleaned/modelSource.csv', index_col=0)

In [372]:
xl1 = xl1.loc[xl1.season==2021]
xl1 = xl1.reset_index(drop=True)
xl1

,season,week,team,player,age,pos,posRank,opponent,passA,passC,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
0,2021.0,1.0,ARI,Kyler Murray,24.0,QB,QB1,TEN,32.0,21.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
1,2021.0,1.0,KCC,Patrick Mahomes,26.0,QB,QB1,CLE,36.0,27.0,...,2.3750,0.6875,0.1250,0.6250,0.1875,0.1250,26.1875,247.6250,110.8125,358.4375
2,2021.0,1.0,NOS,Jameis Winston,27.0,QB,QB1,GBP,20.0,14.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000
3,2021.0,1.0,TBB,Tom Brady,44.0,QB,QB1,DAL,50.0,32.0,...,1.9375,0.6250,0.0000,0.8125,0.0625,0.1875,29.5625,227.6250,158.8125,386.4375
4,2021.0,1.0,SFO,Jared Goff,27.0,QB,QB1,DET,57.0,38.0,...,1.5000,0.4375,0.0625,0.3125,0.2500,0.0625,32.4375,284.8750,134.8750,419.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,2021.0,17.0,MIA,Miami Dolphins,0.0,DF,DF1,TEN,0.0,0.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
5754,2021.0,17.0,DET,Detroit Lions,0.0,DF,DF1,SEA,0.0,0.0,...,2.8750,0.8750,0.1875,0.5000,0.1250,0.0000,23.1875,285.0000,95.5625,380.5625
5755,2021.0,17.0,DEN,Denver Broncos,0.0,DF,DF1,LAC,0.0,0.0,...,1.6875,0.7500,0.0625,0.4375,0.1250,0.1250,26.6250,223.6250,119.8125,343.4375
5756,2021.0,17.0,MIN,Minnesota Vikings,0.0,DF,DF1,GBP,0.0,0.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000


In [373]:
# Define features and labels
y = xl1[[
'passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst'                                 
]]

X = xl1.drop(columns=[
    'season',
    'team',
    'player',
    'opponent',
    'passA',
    'passC',
    'passY',
    'passT',
    'passI',
    'pass2',
    'rushA',
    'rushY',
    'rushT',
    'rush2',
    'recC',
    'recY',
    'recT',
    'rec2',
    'fum',
    'XPA',
    'XPM',
    'FGA',
    'FGM',
    'FG50',
    'defSack',
    'defI',
    'defSaf',
    'defFum',
    'defBlk',
    'defT',
    'defPtsAgainst',
    'defPassYAgainst',
    'defRushYAgainst',
    'defYdsAgainst'
])

droppedCols = xl1[['season',
'team',
'player',
'pos',
'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])
X

,week,age,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,posRank_QB2,posRank_QB3,posRank_RB1,posRank_RB2,posRank_RB3,posRank_TE1,posRank_TE2,posRank_TE3,posRank_WR1,posRank_WR2
0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5754,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5755,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5756,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [374]:
# Insert any missing feature columns
colsFeatures = ['posRank_DF1', 'posRank_PK1', 'posRank_PK2',
       'posRank_PK3', 'posRank_QB1', 'posRank_QB2', 'posRank_QB3',
       'posRank_QB4', 'posRank_RB1', 'posRank_RB2', 'posRank_RB3',
       'posRank_RB4', 'posRank_RB5',
       'posRank_TE1', 'posRank_TE2', 'posRank_TE3', 'posRank_TE4',
       'posRank_TE5', 'posRank_WR1', 'posRank_WR2', 'posRank_WR3',
       ]
colsToAdd = [x for x in colsFeatures if x not in (X.columns)]
X[colsToAdd] = 0
X

,week,age,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,posRank_TE3,posRank_WR1,posRank_WR2,posRank_PK3,posRank_QB4,posRank_RB4,posRank_RB5,posRank_TE4,posRank_TE5,posRank_WR3
0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5754,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5755,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5756,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [375]:
# Put columns in correct order
X = X[['week',
'age',
'passA_curr',
'passC_curr',
'passY_curr',
'passT_curr',
'passI_curr',
'pass2_curr',
'rushA_curr',
'rushY_curr',
'rushT_curr',
'rush2_curr',
'recC_curr',
'recY_curr',
'recT_curr',
'rec2_curr',
'fum_curr',
'XPA_curr',
'XPM_curr',
'FGA_curr',
'FGM_curr',
'FG50_curr',
'defSack_curr',
'defI_curr',
'defSaf_curr',
'defFum_curr',
'defBlk_curr',
'defT_curr',
'defPtsAgainst_curr',
'defPassYAgainst_curr',
'defRushYAgainst_curr',
'defYdsAgainst_curr',
'gamesPlayed_curr',
'gamesPlayed_prior1',
'passA_prior1',
'passC_prior1',
'passY_prior1',
'passT_prior1',
'passI_prior1',
'pass2_prior1',
'rushA_prior1',
'rushY_prior1',
'rushT_prior1',
'rush2_prior1',
'recC_prior1',
'recY_prior1',
'recT_prior1',
'rec2_prior1',
'fum_prior1',
'XPA_prior1',
'XPM_prior1',
'FGA_prior1',
'FGM_prior1',
'FG50_prior1',
'defSack_prior1',
'defI_prior1',
'defSaf_prior1',
'defFum_prior1',
'defBlk_prior1',
'defT_prior1',
'defPtsAgainst_prior1',
'defPassYAgainst_prior1',
'defRushYAgainst_prior1',
'defYdsAgainst_prior1',
'gamesPlayed_prior2',
'passA_prior2',
'passC_prior2',
'passY_prior2',
'passT_prior2',
'passI_prior2',
'pass2_prior2',
'rushA_prior2',
'rushY_prior2',
'rushT_prior2',
'rush2_prior2',
'recC_prior2',
'recY_prior2',
'recT_prior2',
'rec2_prior2',
'fum_prior2',
'XPA_prior2',
'XPM_prior2',
'FGA_prior2',
'FGM_prior2',
'FG50_prior2',
'defSack_prior2',
'defI_prior2',
'defSaf_prior2',
'defFum_prior2',
'defBlk_prior2',
'defT_prior2',
'defPtsAgainst_prior2',
'defPassYAgainst_prior2',
'defRushYAgainst_prior2',
'defYdsAgainst_prior2',
'defSack_curr_opp',
'defI_curr_opp',
'defSaf_curr_opp',
'defFum_curr_opp',
'defBlk_curr_opp',
'defT_curr_opp',
'defPtsAgainst_curr_opp',
'defPassYAgainst_curr_opp',
'defRushYAgainst_curr_opp',
'defYdsAgainst_curr_opp',
'defSack_prior1_opp',
'defI_prior1_opp',
'defSaf_prior1_opp',
'defFum_prior1_opp',
'defBlk_prior1_opp',
'defT_prior1_opp',
'defPtsAgainst_prior1_opp',
'defPassYAgainst_prior1_opp',
'defRushYAgainst_prior1_opp',
'defYdsAgainst_prior1_opp',
'pos_DF',
'pos_PK',
'pos_QB',
'pos_RB',
'pos_TE',
'pos_WR',
'posRank_DF1',
'posRank_PK1',
'posRank_PK2',
'posRank_PK3',
'posRank_QB1',
'posRank_QB2',
'posRank_QB3',
'posRank_RB1',
'posRank_RB2',
'posRank_RB3',
'posRank_TE1',
'posRank_TE2',
'posRank_TE3',
'posRank_WR1',
'posRank_WR2',
'posRank_WR3'
]]


In [376]:
[print(x) for x in X.columns]

week
age
passA_curr
passC_curr
passY_curr
passT_curr
passI_curr
pass2_curr
rushA_curr
rushY_curr
rushT_curr
rush2_curr
recC_curr
recY_curr
recT_curr
rec2_curr
fum_curr
XPA_curr
XPM_curr
FGA_curr
FGM_curr
FG50_curr
defSack_curr
defI_curr
defSaf_curr
defFum_curr
defBlk_curr
defT_curr
defPtsAgainst_curr
defPassYAgainst_curr
defRushYAgainst_curr
defYdsAgainst_curr
gamesPlayed_curr
gamesPlayed_prior1
passA_prior1
passC_prior1
passY_prior1
passT_prior1
passI_prior1
pass2_prior1
rushA_prior1
rushY_prior1
rushT_prior1
rush2_prior1
recC_prior1
recY_prior1
recT_prior1
rec2_prior1
fum_prior1
XPA_prior1
XPM_prior1
FGA_prior1
FGM_prior1
FG50_prior1
defSack_prior1
defI_prior1
defSaf_prior1
defFum_prior1
defBlk_prior1
defT_prior1
defPtsAgainst_prior1
defPassYAgainst_prior1
defRushYAgainst_prior1
defYdsAgainst_prior1
gamesPlayed_prior2
passA_prior2
passC_prior2
passY_prior2
passT_prior2
passI_prior2
pass2_prior2
rushA_prior2
rushY_prior2
rushT_prior2
rush2_prior2
recC_prior2
recY_prior2
recT_prior2
re

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [377]:
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst']
y_pred


,passA,passC,passY,passT,passI,pass2,rushA,rushY,rushT,rush2,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,35.055002,22.754073,264.129619,1.752619,0.766633,0.087833,3.941260,17.773380,0.181121,0.006188,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,38.765888,25.602385,306.170738,2.190593,0.773950,0.127361,2.656549,10.184567,0.118097,0.003718,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,32.841145,20.984987,234.093923,1.698037,0.799377,0.077875,3.204822,13.884066,0.133609,0.010809,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,41.115046,27.398187,326.920804,2.337209,0.892098,0.108531,1.856084,4.957341,0.073857,0.001060,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,37.232983,24.523688,278.333932,1.836335,0.796427,0.151295,2.336575,7.884721,0.082719,0.001729,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.407596,0.870809,0.034647,0.555353,0.078407,0.183534,22.145591,222.977927,113.325662,336.303590
5754,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.234102,0.843564,0.021395,0.540770,0.089427,0.182037,24.656204,248.893020,118.925900,367.818919
5755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.398436,0.839644,0.035337,0.574111,0.071486,0.209778,21.792623,223.859164,110.727701,334.586864
5756,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.118426,0.805167,0.029728,0.493981,0.102725,0.185218,25.035862,241.769986,125.801494,367.571480


In [378]:
# ['passA',0
# 'passC',0
# 'passY',.04
# 'passT',4
# 'passI',-2
# 'pass2',2
# 'rushA',.1
# 'rushY',.1
# 'rushT',6
# 'rush2',2
# 'recC',.25
# 'recY',.1
# 'recT',6
# 'rec2',2
# 'fum',-2
# 'XPA',0
# 'XPM',1
# 'FGA',0
# 'FGM',3
# 'FG50',5
# 'defSack',1
# 'defI',2
# 'defSaf',2
# 'defFum',2
# 'defBlk',1.5
# 'defT',6
# 'defPtsAgainst',0:10, 6:8, 13:7, 17:5, 21:3, 27:2, 34:0, 45:-1, 59:-3, 99:-5
# 'defPassYAgainst',0
# 'defRushYAgainst',0
# 'defYdsAgainst'] 274:5, 324:2, 375:0, 425:-2, 999:-5

multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0
]

In [379]:
# Calculate FANTASY scores
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]

ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]

y['defPtsBin'] = pd.cut(y['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y['defYdsBin'] = pd.cut(y['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)

a = droppedCols.merge(y, left_index=True, right_index=True)
a_pred = droppedCols.merge(y_pred, left_index=True, right_index=True)

a.loc[a['pos']!='DF', 'defPtsBin'] = 0
a.loc[a['pos']!='DF', 'defYdsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0

a = a.drop(columns=['season','team','player','pos','opponent'])
a_pred = a_pred.drop(columns=['season','team','player','pos','opponent'])


/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":


In [380]:

def multer(row):
    return row.multiply(multiplier)

b = a.apply(multer, axis=1)
b = b.apply(np.sum, axis=1)
b = pd.DataFrame(b, columns=['real'])

c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

In [381]:
real_pred = droppedCols.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)
real_pred

,season,team,player,pos,opponent,real,pred
0,2021.0,ARI,Kyler Murray,QB,TEN,34.06,19.024586
1,2021.0,KCC,Patrick Mahomes,QB,CLE,33.78,21.364937
2,2021.0,NOS,Jameis Winston,QB,GBP,30.22,16.886129
3,2021.0,TBB,Tom Brady,QB,DAL,27.16,21.631396
4,2021.0,SFO,Jared Goff,QB,DET,29.22,18.298769
...,...,...,...,...,...,...,...
5753,2021.0,MIA,Miami Dolphins,DF,TEN,3.00,8.548029
5754,2021.0,DET,Detroit Lions,DF,SEA,-7.00,8.271921
5755,2021.0,DEN,Denver Broncos,DF,LAC,0.00,8.662513
5756,2021.0,MIN,Minnesota Vikings,DF,GBP,-6.00,8.041576


In [382]:
season_totals = real_pred.groupby(['player', 'pos']).sum()
season_totals.drop(columns=['season'], inplace=True)
season_totals

,,real,pred
player,pos,,
A.J. Brown,WR,120.05,125.671525
A.J. Green,WR,113.00,104.374511
AJ Dillon,RB,170.45,137.507768
Aaron Jones,RB,207.10,187.050954
Aaron Rodgers,QB,321.53,281.584206
...,...,...,...
Zach Pascal,WR,65.25,86.235297
Zach Wilson,QB,144.68,156.919190
Zack Moss,RB,95.15,103.660287


In [383]:
season_totals['diff'] = season_totals.real.sub(season_totals.pred).abs()
season_totals

,,real,pred,diff
player,pos,,,
A.J. Brown,WR,120.05,125.671525,5.621525
A.J. Green,WR,113.00,104.374511,8.625489
AJ Dillon,RB,170.45,137.507768,32.942232
Aaron Jones,RB,207.10,187.050954,20.049046
Aaron Rodgers,QB,321.53,281.584206,39.945794
...,...,...,...,...
Zach Pascal,WR,65.25,86.235297,20.985297
Zach Wilson,QB,144.68,156.919190,12.239190
Zack Moss,RB,95.15,103.660287,8.510287


In [384]:
season_totals.sort_values(by='diff', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Cooper Kupp,WR,309.70,228.986804,80.713196
Dallas Cowboys,DF,212.00,137.686817,74.313183
Jonathan Taylor,RB,364.55,294.863248,69.686752
Josh Allen,QB,390.42,322.207005,68.212995
Ja'Marr Chase,WR,241.45,175.548743,65.901257
James Conner,RB,225.85,165.131006,60.718994
Jalen Hurts,QB,326.06,265.991032,60.068968
New York Jets,DF,61.00,119.949750,58.949750
Matt Ryan,QB,215.98,274.148270,58.168270


In [385]:
season_totals.sort_values(by='real', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Josh Allen,QB,390.42,322.207005,68.212995
Jonathan Taylor,RB,364.55,294.863248,69.686752
Justin Herbert,QB,359.44,319.074116,40.365884
Tom Brady,QB,352.50,307.603013,44.896987
Patrick Mahomes,QB,341.16,322.859937,18.300063
Jalen Hurts,QB,326.06,265.991032,60.068968
Aaron Rodgers,QB,321.53,281.584206,39.945794
Joe Burrow,QB,318.24,296.056195,22.183805
Matthew Stafford,QB,315.42,267.139732,48.280268


In [387]:
season_totals.sort_values(by='pred', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Patrick Mahomes,QB,341.16,322.859937,18.300063
Josh Allen,QB,390.42,322.207005,68.212995
Justin Herbert,QB,359.44,319.074116,40.365884
Tom Brady,QB,352.50,307.603013,44.896987
Joe Burrow,QB,318.24,296.056195,22.183805
Jonathan Taylor,RB,364.55,294.863248,69.686752
Dak Prescott,QB,293.56,290.490391,3.069609
Derek Carr,QB,245.52,290.149844,44.629844
Aaron Rodgers,QB,321.53,281.584206,39.945794


In [388]:
season_totals.query('pos == "RB"').sort_values(by='pred', ascending=False).head(10)


,,real,pred,diff
player,pos,,,
Jonathan Taylor,RB,364.55,294.863248,69.686752
Austin Ekeler,RB,285.15,247.226621,37.923379
Najee Harris,RB,268.30,244.766689,23.533311
Joe Mixon,RB,285.60,228.918934,56.681066
Dalvin Cook,RB,195.85,221.456500,25.606500
Ezekiel Elliott,RB,229.46,220.930182,8.529818
Nick Chubb,RB,215.95,217.210979,1.260979
Alvin Kamara,RB,203.75,209.346961,5.596961
Antonio Gibson,RB,199.95,198.852747,1.097253


In [389]:
real_pred2 = real_pred.merge(y_pred, left_index=True, right_index=True)
real_pred2.loc[real_pred2.player=='Cooper Kupp']

,season,team,player,pos,opponent,real,pred,passA,passC,passY,...,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst,defPtsBin,defYdsBin
1988,2021.0,LAR,Cooper Kupp,WR,CHI,18.55,12.261092,0.009750,0.005512,0.123495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2098,2021.0,LAR,Cooper Kupp,WR,IND,30.15,14.479917,0.013473,0.006257,0.113378,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2217,2021.0,LAR,Cooper Kupp,WR,TBB,23.85,13.889055,0.018767,0.010459,0.253604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2379,2021.0,LAR,Cooper Kupp,WR,ARI,7.65,13.435164,0.020270,0.008986,0.168037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2475,2021.0,LAR,Cooper Kupp,WR,SEA,10.95,14.257078,0.017067,0.006564,0.113129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2573,2021.0,LAR,Cooper Kupp,WR,NYG,27.25,14.237954,0.017409,0.008195,0.134386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2675,2021.0,LAR,Cooper Kupp,WR,DET,30.10,16.737330,0.051909,0.005675,0.075816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2776,2021.0,LAR,Cooper Kupp,WR,HOU,19.25,15.398634,0.018148,0.006772,0.108136,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2894,2021.0,LAR,Cooper Kupp,WR,TEN,12.25,14.569005,0.024059,0.008910,0.144583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5
2989,2021.0,LAR,Cooper Kupp,WR,SFO,14.95,14.820084,0.020516,0.008617,0.139409,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,5


In [ ]:

# Try ML model with positions split out ahead of time
# Deal with currentstats throwing things out of whack before season starts
# Add in Punt- and Kick-Return scores